In [13]:
library(dplyr)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [5]:
# reading data for a specific case where: tree.type=='6_taxa.nzero_2', internal.el==0.1, test=='AU_true'
# since we had a 1000 simulations, and 6 taxa has 105 possible trees, we expect 105000 rows

df_tmp <- read.csv(file="df_tmp.csv", 
                   header=TRUE,
                   sep=",")
                   
nrow(df_tmp)

[1] 105000

In [6]:
tail(df_tmp,10)

,taxa,sim.index,tree.index,tree.type,internal.el,nsim,instanceRun,p.value,test,true.set
,<int>,<int>,<int>,<fct>,<dbl>,<int>,<dbl>,<dbl>,<fct>,<int>
104991,6,1000,96,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.570,AU_true,1
104992,6,1000,97,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0
104993,6,1000,98,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0
104994,6,1000,99,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0
104995,6,1000,100,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0
104996,6,1000,101,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0
104997,6,1000,102,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.237,AU_true,1
104998,6,1000,103,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0
104999,6,1000,104,6_taxa.nzero_2,0.1,1000,2.018111e+13,0.000,AU_true,0


In [21]:
nsim <- mean(df_tmp$nsim) # extracting number of simulations from data
taxa <- mean(df_tmp$taxa) # extracting the taxa (6 or 8)
ntrue <- sum(df_tmp$true.set) # extracting number of true trees

# setting the probability we want above which a tree will be included in the confidence set (CS)
probability.threshold <- 0.05 

Define $$p_j= \text{long-run proportion of } C_b \text{ with } j\in C_b \tag{1}$$

In [8]:
# adding a column for the individual tree index probability to be in the CS
df_tmp$p.i <- as.numeric(df_tmp$p.value > probability.threshold)

In [42]:
# defining a function to determine the number of possible trees, for later iterations:
number_of_possible_trees <- function(taxa){
    b <- factorial(2*taxa - 5)/(factorial(taxa - 3)*2^(taxa - 3))
    return(b)
}

number_of_possible_trees(taxa)

[1] 105

In [11]:
# preparing for loop:
upper_limit <- number_of_possible_trees(taxa)

cov_tmp <- data.frame(
  tree.type=character(),
  internal.el=integer(),
  test=character(),
  tree_i=integer(), 
  tree_j=integer(),
  p.i=double(),
  p.j=double(),
  p.ij=double(),
  stringsAsFactors=FALSE
)

$$p_{ij}= \text{long-run proportion of } C_b \text{ with both } i,j\in C_b \tag{2}$$

In [14]:
# main loop to check for correlation between every combination of trees:
k <- 1
for (tree_i in c(1:(upper_limit-1))){
  lower_limit <- tree_i + 1
  tmp.p.i <- sum(df_tmp %>% filter(tree.index==tree_i) %>% select(p.i))/nsim
  for (tree_j in c(lower_limit:upper_limit)){
    tmp.p.j <- sum(df_tmp %>% filter(tree.index==tree_j) %>% select(p.i))/nsim
    tmp.p.ij <- as.double((df_tmp %>% 
                    filter((tree.index==tree_i & p.i == 1) | (tree.index==tree_j & p.i == 1)) %>% 
                    group_by(sim.index) %>% 
                    summarize(n=n()) %>%
                    # already filtered out on either one of them in the set, so 0 isn't possible, 
                    # so we're safe to assume the possible values here are 1 or 2. 
                    # Using a minus 1 logic to determine of they're both in the set:
                    mutate(n=n-1) %>% summarize(sum(n)))/nsim) 
    cov_tmp[k,] <- list('6_taxa.nzero_2', 0.1, 'AU_true', tree_i, tree_j, tmp.p.i, tmp.p.j, tmp.p.ij)
    k <- k + 1
  }
}

In [15]:
# we expect to find 105 choose 2 rows = 5460
nrow(cov_tmp)

[1] 5460

In [18]:
# spot checking data
head(cov_tmp[cov_tmp$p.i > 0,], 15)

,tree.type,internal.el,test,tree_i,tree_j,p.i,p.j,p.ij
,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>
4515,6_taxa.nzero_2,0.1,AU_true,62,63,0.998,0.000,0.000
4516,6_taxa.nzero_2,0.1,AU_true,62,64,0.998,0.999,0.997
4517,6_taxa.nzero_2,0.1,AU_true,62,65,0.998,1.000,0.998
4518,6_taxa.nzero_2,0.1,AU_true,62,66,0.998,0.000,0.000
4519,6_taxa.nzero_2,0.1,AU_true,62,67,0.998,1.000,0.998
4520,6_taxa.nzero_2,0.1,AU_true,62,68,0.998,0.000,0.000
4521,6_taxa.nzero_2,0.1,AU_true,62,69,0.998,0.000,0.000
4522,6_taxa.nzero_2,0.1,AU_true,62,70,0.998,0.000,0.000
4523,6_taxa.nzero_2,0.1,AU_true,62,71,0.998,0.000,0.000


Then the covariance 


$$Cov[\delta_{jb},\delta_{ib}]=p_{ij}-p_ip_j \tag{3}$$

In [19]:
cov_tmp <- cov_tmp %>% mutate(cov = p.ij - p.i*p.j)

$$Var[\delta_{jB}]=p_j(1-p_j) \tag{4}$$

In [20]:
cov_tmp <- cov_tmp %>% mutate(var.i = p.i*(1 - p.i))

In [47]:
# spot checking data
head(cov_tmp[cov_tmp$p.i > 0,], 15)

,tree.type,internal.el,test,tree_i,tree_j,p.i,p.j,p.ij,cov,var.i
,<chr>,<dbl>,<chr>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
4515,6_taxa.nzero_2,0.1,AU_true,62,63,0.998,0.000,0.000,0e+00,0.001996
4516,6_taxa.nzero_2,0.1,AU_true,62,64,0.998,0.999,0.997,-2e-06,0.001996
4517,6_taxa.nzero_2,0.1,AU_true,62,65,0.998,1.000,0.998,0e+00,0.001996
4518,6_taxa.nzero_2,0.1,AU_true,62,66,0.998,0.000,0.000,0e+00,0.001996
4519,6_taxa.nzero_2,0.1,AU_true,62,67,0.998,1.000,0.998,0e+00,0.001996
4520,6_taxa.nzero_2,0.1,AU_true,62,68,0.998,0.000,0.000,0e+00,0.001996
4521,6_taxa.nzero_2,0.1,AU_true,62,69,0.998,0.000,0.000,0e+00,0.001996
4522,6_taxa.nzero_2,0.1,AU_true,62,70,0.998,0.000,0.000,0e+00,0.001996
4523,6_taxa.nzero_2,0.1,AU_true,62,71,0.998,0.000,0.000,0e+00,0.001996


$$\sum_{j=1}^K Var[\delta_{jb}]$$

In [53]:
var.delta <- cov_tmp %>% group_by(tree_i) %>% summarize(var.delta=mean(var.i)) %>% summarize(sum(var.delta))

var.delta

sum(var.delta)
<dbl>
0.007978


$$Var\Big[\sum_{j=1}^K\delta_{jb}\Big]=\sum_{j=1}^K Var[\delta_{jb}] + 2\sum_{i<j}Cov[\delta_{jb}, \delta_{ib}] \tag{5}$$


$$Var[C]=Var\Big[\sum_{j=1}^K\delta_{jb}\Big]/(K^2B) \tag{6}$$

In [59]:
var.C <- as.double(( var.delta + 2*sum(cov_tmp$cov) )/(nsim*ntrue^2))

var.C

[1] 1.47358e-13

Let $C$ be the average coverage, then
$$C=\sum_{j=1}^K\sum_{\delta=1}^B\delta_{jb}/(KB)$$ 

In [60]:
C <- sum(df_tmp[df_tmp$true.set==1, 'p.i'])/sum(df_tmp$true.set) # the denominator here = 9000 includes both K*B

C

[1] 0.9991111

Logistic transformation:
$$g(C)=\log{\Big[\frac{C}{1-C}\Big]}=\log{[C]}-\log{[1-C]}$$

In [61]:
g.C <- log(1/C) + log(1/(1-C))  

g.C

[1] 7.026428

$$g^\prime(C)=\frac{1}{C} + \frac{1}{1-C} \tag{7}$$

In [62]:
g.prime.C <- 1/C + 1/(1-C)  
  
g.prime.C

[1] 1126.001

$$Var[g(C)]={g^\prime(C)}^2 \cdot Var[C] \tag{8}$$

In [63]:
var.g.c <- g.prime.C^2 * var.C
  
var.g.c

[1] 1.86832e-07

Let $[L, U]$ be the lower and upper limits of the 95% CI for $g(C)$, then:
$$[L, U]=g(C)\pm 1.96 \sqrt{Var[g(C)]} \tag{9}$$

In [64]:
L.U <- c(g.C + 1.96*sqrt(var.g.c), g.C - 1.96*sqrt(var.g.c))  
  
L.U

[1] 7.027275 7.025580

Then the 95% CI for C is:


$$[g^{-1}(L), g^{-1}(U)] \tag{10}$$


In [65]:
CI <- exp(L.U)

CI

[1] 1126.955 1125.047